# 문제 09 유형(DataSet_09.csv 이용)

 구분자 : comma(“,”), 2000 Rows, 16 Columns, UTF-8 인코딩

 항공사에서 고객만족도 조사를 하고 서비스 개선에 활용하고자 아래와 같은 데이터를 준비하였다.

 컬 럼 / 정 의 / Type
 satisfaction / 서비스 만족 여부 / String
 Gender / 성별 / String
 Age / 나이 / Double
 Customer_Type / 고객 타입 / String
 Class / 탑승 좌석 등급 / String
 Flight_Distance / 비행 거리 / Double
 Seat_comfort / 좌석 안락도 점수 / Double
 Food_and_Drink / 식사와 음료 점수 / Double
 Inflight_wifi_service / 기내 와이파이 서비스 점수 / Double
 Inflight_entertainment / 기내 엔터테인먼트 서비스 점수 / Double
 Onboard_service / 탑승 서비스 점수 / Double
 Leg_room_service / 다리 공간 점수 / Double
 Baggage_handling / 수하물 취급 점수 / Double
 Cleanliness / 청결도 점수 / Double
 Departure_Daly_in_Minutes / 출발 지연 (분) / Double
 Arrival_Delay_in_Minutes / 도착 지연 (분) / Double

In [2]:
import pandas as pd

data9=pd.read_csv('C:/Users/joey0/CodeStudy/proDS/Dataset/Dataset_09.csv')

## 1.데이터 타입을 위 표에 정의된 타입으로 전처리를 한 후, 데이터 파일 내에 결측값은 총 몇 개인가?

In [3]:
data9.isna().sum().sum()

#정답 : 5

5

## 2.다음에 제시된 데이터 처리를 하고 카이제곱 독립성 검정을 수행하시오.
 - 결측값이 있다면 해당 행을 제거하시오.
 - 나이는 20 이하이면 10, 30 이하이면 20, 40 이하이면 30, 50 이하이면 40, 60 이하이면 50, 60 초과는 60으로 변환하여 Age_gr으로 파생변수를 생성하시오.
 - Age_gr, Gender, Customer_Type, Class 변수가 satisfaction에 영향이 있는지 카이제곱 독립성 검정을 수행하시오. 
 - 연관성이 있는 것으로 파악된 변수의 검정통계량 추정치를 정수 부분만 기술하시오.

In [4]:
q2=data9.copy()
q2.columns

import numpy as np
q2['Age_gr']=np.where(q2.Age <= 20, 10,
                np.where(q2.Age <= 30, 20,
                   np.where(q2.Age <= 40, 30,
                     np.where(q2.Age <= 50, 40,
                        np.where(q2.Age <= 60, 50,  60)))))


from scipy.stats import chi2_contingency

In [5]:
# - Age_gr, Gender, Customer_Type, Class 변수가 satisfaction에 영향이 있는지
var_list=['Age_gr', 'Gender', 'Customer_Type', 'Class']

q2_out=[]
for i in var_list:
    q2_tab=pd.crosstab(index=q2[i], columns=q2['satisfaction'])
    chi, pvalue, *_=chi2_contingency(q2_tab)
    q2_out.append([i, chi, pvalue])

q2_out=pd.DataFrame(q2_out, columns=['var', 'chi', 'pvalue'])

q2_out[q2_out.pvalue < 0.05]['chi']

#정답 : 1068

1    1068.632582
Name: chi, dtype: float64

## 3.고객 만족도를 라벨로 하여 다음과 같이 로지스틱 회귀분석을 수행하시오. 
 - 결측치가 포함된 행은 제거
 - 데이터를 7대 3으로 분리 (Seed = 123)
 - 아래의 11개 변수를 Feature로 사용
 Flight_Distance, Seat_comfort, Food_and_drink, Inflight_wifi_service, 
 Inflight_entertainment,Onboard_service, Leg_room_service, Baggage_handling,
 Cleanliness, Departure_Delay_in_Minutes, Arrival_Delay_in_Minutes
 
 - Seed = 123, 이외의 항목은 모두 Default 사용
 - 예측 정확도를 측정하고 dissatisfied의 f1 score를 소수점 넷째 자리에서 반올림하여 소수점 셋째 자리까지 기술하시오.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# from sklearn import metrics
from sklearn.metrics import f1_score, classification_report


x_var=['Flight_Distance', 'Seat_comfort', 'Food_and_drink',
       'Inflight_wifi_service','Inflight_entertainment', 'Onboard_service', 
       'Leg_room_service', 'Baggage_handling', 'Cleanliness',
       'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']

q3=data9.dropna()

X_train, X_test, y_train, y_test = \
    train_test_split(q3[x_var], q3['satisfaction'], 
                 test_size = 0.3, random_state = 123)

model = LogisticRegression(solver = 'liblinear',random_state=123)
result = model.fit(X_train, y_train)

y_pred = model.predict(X_test)


f1_score(y_test, y_pred, pos_label='dissatisfied')

#정답 : 0.777

0.7771084337349397